In [273]:
# 패키지 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBRegressor
import xgboost
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.size'] = 15
import warnings 
warnings.filterwarnings(action='ignore')

In [274]:
EDA = pd.read_csv('EDA.csv',encoding='cp949')

In [275]:
EDA.head()

,gid,geometry,four,six,seven,2018,2019,2020,2021,seventeen,eighteen,nineteen,twenty,twentyone,two,cctv
0,마라231909,MULTIPOLYGON (((128.8523707124823 35.205903552...,10,2,0,0,0,0,0,0,0,0,0,0,0,0
1,마라154986,MULTIPOLYGON (((128.7688788763629 35.276233217...,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,마라213944,MULTIPOLYGON (((128.833116749215 35.2376741072...,9,0,0,0,0,0,0,0,0,0,0,0,0,0
3,마마105025,MULTIPOLYGON (((128.7155376522062 35.311944657...,0,0,1,0,0,0,0,0,0,0,0,0,0,3
4,마라202940,MULTIPOLYGON (((128.8209718821295 35.234200772...,48,2,0,0,0,0,0,0,0,0,1,0,0,0


In [276]:
EDA1 = EDA[EDA['cctv'] >= 1]

In [277]:
EDA1.head()

,gid,geometry,four,six,seven,2018,2019,2020,2021,seventeen,eighteen,nineteen,twenty,twentyone,two,cctv
3,마마105025,MULTIPOLYGON (((128.7155376522062 35.311944657...,0,0,1,0,0,0,0,0,0,0,0,0,0,3
6,마라142954,MULTIPOLYGON (((128.7552421328997 35.247523262...,21,4,0,0,0,0,0,0,0,0,0,0,0,1
20,마라281955,MULTIPOLYGON (((128.9080022709618 35.246743775...,397,6,1,176,156,122,125,1,0,0,0,0,0,5
22,마라273935,MULTIPOLYGON (((128.8989018157547 35.228816966...,234,0,1,306,312,247,213,0,0,0,0,0,0,4
26,마라186890,MULTIPOLYGON (((128.8026719911344 35.189317442...,15,5,1,0,0,0,0,0,0,0,0,0,0,3


# LGBM

In [278]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso
from sklearn.ensemble import  RandomForestRegressor,ExtraTreesRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error

In [279]:
import lightgbm as lgb

In [280]:
x_train = EDA1[['six', 'seven', '2018', '2019', '2020', '2021','seventeen','eighteen','nineteen','twenty','twentyone','two','cctv']]
y_train = EDA1[['four']]
x_test  = EDA[['six', 'seven', '2018', '2019', '2020', '2021','seventeen','eighteen','nineteen','twenty','twentyone','two','cctv']]

In [281]:
dev_x, val_x, dev_y, val_y = train_test_split(x_train, y_train, test_size = 0.2, random_state = 2022)

In [282]:
params = {
    'objective' : 'regression',
    'metric' : 'mae',
    'num_leaves' : 40,
    'learning_rate' : 0.004,
    'bagging_fraction' : 0.6,
    'feature_fraction' : 0.6,
    'bagging_frequnecy' : 6,
    'bagging_seed' : 2021,
    'verbosity' : -1,
    'seed' : 2021
}

In [283]:
lgtrain = lgb.Dataset(dev_x,dev_y)
lgval = lgb.Dataset(val_x,val_y)
evals_result = {}
model1 = lgb.train(params = params,
                  train_set = lgtrain,
                  num_boost_round = 5000,
                  valid_sets = [lgtrain,lgval],
                  early_stopping_rounds = 100,
                  verbose_eval = 150,
                  evals_result = evals_result)

[LightGBM] [Warning] Unknown parameter: bagging_frequnecy
Training until validation scores don't improve for 100 rounds
[150]	training's l1: 70.6508	valid_1's l1: 66.2393
[300]	training's l1: 61.6708	valid_1's l1: 60.1026
[450]	training's l1: 57.7732	valid_1's l1: 56.5905
[600]	training's l1: 55.6866	valid_1's l1: 54.8284
[750]	training's l1: 54.2493	valid_1's l1: 54.1393
[900]	training's l1: 53.1808	valid_1's l1: 54.0773
Early stopping, best iteration is:
[849]	training's l1: 53.5484	valid_1's l1: 54.0402


In [284]:
pred1 = model1.predict(x_test,num_iteration=model1.best_iteration)

In [285]:
print("MAE:",mean_absolute_error(y_test, pred1))

MAE: 42.72216890837733


# RandomForest

In [291]:
from sklearn.ensemble import RandomForestRegressor

In [292]:
model2 = RandomForestRegressor(n_jobs=-1, random_state=42)

In [293]:
model2.fit(x_train, y_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [294]:
pred2 = model2.predict(x_test)

In [295]:
print("MAE:",mean_absolute_error(y_test, pred2))

MAE: 55.78079255380983


# Catboost

In [296]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings, random
warnings.filterwarnings(action='ignore')

from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold

from sklearn.cluster import KMeans
from catboost import CatBoostClassifier, Pool

In [297]:
x_train = EDA1[['six', 'seven', '2018', '2019', '2020', '2021','seventeen','eighteen','nineteen','twenty','twentyone','two','cctv']]
y_train = EDA1[['four']]
x_test  = EDA[['six', 'seven', '2018', '2019', '2020', '2021','seventeen','eighteen','nineteen','twenty','twentyone','two','cctv']]
y_test = EDA['four']

In [298]:
from catboost import CatBoostRegressor

In [300]:
cb_model = CatBoostRegressor(
    iterations = 500,
    learning_rate = 0.05,
    depth = 10,
    eval_metric = 'MAE',
    random_seed = 2022,
    bagging_temperature = 0.2,
    od_type = 'Iter',
    metric_period = 50,
    od_wait = 20
)

In [301]:
dev_x, val_x, dev_y, val_y = train_test_split(x_train, y_train, test_size = 0.2, random_state = 2021)

In [302]:
cb_model.fit(
    dev_x,dev_y,
    eval_set = (val_x,val_y),
    use_best_model = True,
    verbose = 50
)

0:	learn: 86.0245978	test: 89.2686849	best: 89.2686849 (0)	total: 5.47ms	remaining: 2.73s
50:	learn: 61.9222664	test: 72.1740966	best: 72.1740966 (50)	total: 209ms	remaining: 1.84s
100:	learn: 50.4029342	test: 63.7880746	best: 63.7566616 (97)	total: 420ms	remaining: 1.66s
150:	learn: 42.4935884	test: 58.9155718	best: 58.8954058 (149)	total: 726ms	remaining: 1.68s
200:	learn: 37.2495523	test: 57.6739146	best: 57.6534686 (197)	total: 972ms	remaining: 1.45s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 57.65346864
bestIteration = 197

Shrink model to first 198 iterations.


In [303]:
pred3 = cb_model.predict(x_test)

In [304]:
print("MAE:",mean_absolute_error(y_test, pred3))

MAE: 52.21572282759797


# XGBoost

In [305]:
params = {
    'objective' : 'reg:linear',
    'eval_metric' : 'mae',
    'eta' : 0.001,
    'max_depth' : 10,
    'subsample' : 0.6,
    'colsample_bytree' : 0.6,
    'alpha' : 0.001,
    'random_state' : 2021,
    'silent' : True
}


In [307]:
dev_x, val_x, dev_y, val_y = train_test_split(x_train, y_train, test_size = 0.2, random_state = 2022)

In [308]:
tr_data = xgb.DMatrix(dev_x,dev_y)
va_data = xgb.DMatrix(val_x,val_y)

watchlist = [(tr_data,'train'), (va_data,'valid')]

model_xgb = xgb.train(
    params = params,
    dtrain = tr_data,
    num_boost_round = 2000,
    evals = watchlist,
    maximize = False,
    early_stopping_rounds = 100,
    verbose_eval = 100
)

[0]	train-mae:93.91775	valid-mae:80.08653
[100]	train-mae:86.99316	valid-mae:73.82806
[200]	train-mae:80.92593	valid-mae:68.48614
[300]	train-mae:75.77408	valid-mae:64.26708
[400]	train-mae:71.22688	valid-mae:61.24394
[500]	train-mae:67.14778	valid-mae:58.75109
[600]	train-mae:63.59923	valid-mae:56.91884
[700]	train-mae:60.47940	valid-mae:55.45323
[800]	train-mae:57.70274	valid-mae:54.34262
[900]	train-mae:55.26190	valid-mae:53.61702
[1000]	train-mae:53.08186	valid-mae:53.11166
[1100]	train-mae:51.17434	valid-mae:52.78200
[1200]	train-mae:49.43947	valid-mae:52.54643
[1300]	train-mae:47.88657	valid-mae:52.34921
[1400]	train-mae:46.51078	valid-mae:52.23352
[1500]	train-mae:45.26772	valid-mae:52.11384
[1600]	train-mae:44.14037	valid-mae:52.05564
[1700]	train-mae:43.14108	valid-mae:51.95044
[1800]	train-mae:42.26803	valid-mae:51.92527
[1900]	train-mae:41.36128	valid-mae:51.89595
[1999]	train-mae:40.53746	valid-mae:51.88799


In [309]:
dtest = xgb.DMatrix(x_test)

In [310]:
pred4 = model_xgb.predict(dtest,ntree_limit=model_xgb.best_ntree_limit)

In [311]:
print("MAE:",mean_absolute_error(y_test, pred4))

MAE: 34.13679312732092


In [312]:
EDA['pred']=pred4

In [313]:
EDA.head()

,gid,geometry,four,six,seven,2018,2019,2020,2021,seventeen,eighteen,nineteen,twenty,twentyone,two,cctv,pred
0,마라231909,MULTIPOLYGON (((128.8523707124823 35.205903552...,10,2,0,0,0,0,0,0,0,0,0,0,0,0,20.891960
1,마라154986,MULTIPOLYGON (((128.7688788763629 35.276233217...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28.464565
2,마라213944,MULTIPOLYGON (((128.833116749215 35.2376741072...,9,0,0,0,0,0,0,0,0,0,0,0,0,0,28.464565
3,마마105025,MULTIPOLYGON (((128.7155376522062 35.311944657...,0,0,1,0,0,0,0,0,0,0,0,0,0,3,43.803875
4,마라202940,MULTIPOLYGON (((128.8209718821295 35.234200772...,48,2,0,0,0,0,0,0,0,0,1,0,0,0,29.923029


In [314]:
EDA2 = EDA.sort_values(by=['pred'],ascending=False)

In [315]:
EDA2.head()

,gid,geometry,four,six,seven,2018,2019,2020,2021,seventeen,eighteen,nineteen,twenty,twentyone,two,cctv,pred
488,마라258935,MULTIPOLYGON (((128.882422588875 35.2290063205...,4998,2,0,93,88,98,104,0,0,0,16,0,0,2,2327.364502
2172,마라243939,MULTIPOLYGON (((128.8660036604245 35.232799242...,1880,10,1,82,78,86,86,0,0,0,21,0,0,5,1305.710815
5695,마라247971,MULTIPOLYGON (((128.8708839231963 35.261595912...,549,0,0,72,64,51,51,0,0,0,18,0,0,0,857.494446
7524,마라247970,MULTIPOLYGON (((128.8708687392403 35.260694466...,827,1,0,0,0,0,0,0,0,0,32,0,0,0,780.400146
6550,마라243938,MULTIPOLYGON (((128.865988546614 35.2318977900...,1347,0,1,7,6,6,6,0,0,0,15,1,0,1,743.605103


In [316]:
EDA2.to_csv('EDA2.csv')